In [3]:
# Log Transformation yapılabilir.
# total_cases sağa çarpık (skewed) → log bunu düzeltir
# XGBoost ve RandomForest log dönüşümüyle daha stabil öğrenir
# Büyük vakalar (100–400) çok ağırlık bindirmesin diye log dengeler
import os, sys

# notebook içindeyken proje köküne çık
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)


PROJECT_ROOT: c:\Users\MONSTER\Desktop\YILBAŞI PROJEM - Kopya


In [8]:
import pandas as pd
import numpy as np

# 1) Veriyi yükle (Data/ yok, direkt dosya isimleri)
train_features = pd.read_csv("Data/dengue_features_train.csv")
train_labels = pd.read_csv("Data/dengue_labels_train.csv")
test_features = pd.read_csv("Data/dengue_features_test.csv")

# 2) Feature + label tek tabloda birleştir
df = train_features.merge(train_labels, on=["city", "year", "weekofyear"])

# 3) Şehir bazında ayır, zaman sırasına göre sırala
df_sj = df[df["city"] == "sj"].sort_values(["year", "weekofyear"]).reset_index(drop=True)
df_iq = df[df["city"] == "iq"].sort_values(["year", "weekofyear"]).reset_index(drop=True)

# 4) Eksikleri forward-fill + backward-fill ile doldur
df_sj = df_sj.fillna(method="ffill").fillna(method="bfill")
df_iq = df_iq.fillna(method="ffill").fillna(method="bfill")

# 5) Tek bir temiz dataframe isterse tekrar birleştir
df_clean = pd.concat([df_sj, df_iq], axis=0).sort_values(["year", "weekofyear"])
df_clean = df_clean.reset_index(drop=True)

df_clean.head()


C:\Users\MONSTER\AppData\Local\Temp\ipykernel_16172\1568005869.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_sj = df_sj.fillna(method="ffill").fillna(method="bfill")
C:\Users\MONSTER\AppData\Local\Temp\ipykernel_16172\1568005869.py:18: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_iq = df_iq.fillna(method="ffill").fillna(method="bfill")


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [9]:
# 1, 2, 3 hafta gecikmeli vaka sayıları
# Dengue bir zaman serisi ve bugünkü vaka sayısını en çok:
# 1 hafta önceki vaka
# 2 hafta önceki vaka
# 3 hafta önceki vaka
# belirliyor. O yüzden bunları feature olarak eklemek puanı ciddi düşürüyor.
for lag in [1, 2, 3]:
    df_sj[f"cases_lag_{lag}"] = df_sj["total_cases"].shift(lag)
    df_iq[f"cases_lag_{lag}"] = df_iq["total_cases"].shift(lag)

df_sj[["total_cases", "cases_lag_1", "cases_lag_2", "cases_lag_3"]].head(10)


,total_cases,cases_lag_1,cases_lag_2,cases_lag_3
0,4,NaN,NaN,NaN
1,5,4.0,NaN,NaN
2,4,5.0,4.0,NaN
3,3,4.0,5.0,4.0
4,6,3.0,4.0,5.0
5,2,6.0,3.0,4.0
6,4,2.0,6.0,3.0
7,5,4.0,2.0,6.0
8,10,5.0,4.0,2.0
9,6,10.0,5.0,4.0


In [10]:
df_sj = df_sj.dropna().reset_index(drop=True)
df_iq = df_iq.dropna().reset_index(drop=True)

df_sj.head()
df_iq.head()
#NaN olanları düşürüyoruz çünkü lag feature'lar nedeniyle başta NaN oluştu

,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases,cases_lag_1,cases_lag_2,cases_lag_3
0,iq,2000,29,2000-07-22,0.227729,0.145429,0.254200,0.200314,5.60,295.357143,...,9.114286,25.766667,10.533333,31.5,14.7,30.0,0,0.0,0.0,0.0
1,iq,2000,30,2000-07-29,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,...,9.500000,26.600000,11.480000,33.3,19.1,4.0,0,0.0,0.0,0.0
2,iq,2000,31,2000-08-05,0.205529,0.190757,0.231671,0.255314,16.24,297.191429,...,13.771429,25.340000,10.940000,32.0,17.0,11.5,0,0.0,0.0,0.0
3,iq,2000,32,2000-08-12,0.312486,0.329986,0.380586,0.387271,89.37,297.320000,...,11.471429,27.016667,11.650000,34.0,19.9,72.9,0,0.0,0.0,0.0
4,iq,2000,33,2000-08-19,0.384133,0.392240,0.341780,0.382750,42.08,297.627143,...,13.700000,26.583333,10.316667,33.0,20.5,50.1,0,0.0,0.0,0.0


In [11]:
# Şimdi sıradaki adım → Rolling (hareketli ortalama) feature’ları eklemek.
# Sivrisinek popülasyonu ani bile değişmez, birkaç haftalık birikim etkisi vardır.
# "Son 3 haftanın ortalama sıcaklığı" → tek bir haftadan çok daha iyi bilgi verir.
# Kaggle’daki en iyi çözümlerde mutlaka rolling feature var.

In [12]:
# --- SAN JUAN (SJ) ROLLING FEATURES ---

rolling_cols = [
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "station_avg_temp_c",
    "reanalysis_max_air_temp_k",
]

df_sj = df_clean[df_clean["city"] == "sj"].copy()

# Rollings
for col in rolling_cols:
    df_sj[f"{col}_roll3"] = df_sj[col].rolling(window=3).mean()
    df_sj[f"{col}_roll5"] = df_sj[col].rolling(window=5).mean()

# Lag features (zaten eklemiştin ama tekrar burada yer alsın diye)
df_sj["cases_lag_1"] = df_sj["total_cases"].shift(1)
df_sj["cases_lag_2"] = df_sj["total_cases"].shift(2)
df_sj["cases_lag_3"] = df_sj["total_cases"].shift(3)

# NaN temizliği
df_sj = df_sj.dropna().reset_index(drop=True)

df_sj.head()


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_specific_humidity_g_per_kg_roll5,reanalysis_dew_point_temp_k_roll3,reanalysis_dew_point_temp_k_roll5,station_avg_temp_c_roll3,station_avg_temp_c_roll5,reanalysis_max_air_temp_k_roll3,reanalysis_max_air_temp_k_roll5,cases_lag_1,cases_lag_2,cases_lag_3
0,sj,1990,22,1990-05-28,0.19620,0.262200,0.251200,0.247340,7.52,299.518571,...,16.023429,295.521905,294.586286,27.709524,27.057143,301.266667,300.90,3.0,4.0,5.0
1,sj,1990,23,1990-06-04,0.19620,0.174850,0.254314,0.181743,9.58,299.630000,...,16.663429,295.660952,295.273714,28.176190,27.591429,301.900000,301.42,6.0,3.0,4.0
2,sj,1990,24,1990-06-11,0.11290,0.092800,0.205071,0.210271,3.48,299.207143,...,17.035714,295.846190,295.656571,28.157143,27.731429,301.866667,301.50,2.0,6.0,3.0
3,sj,1990,25,1990-06-18,0.07250,0.072500,0.151471,0.133029,151.12,299.591429,...,17.261429,296.082857,295.876000,27.966667,28.062857,301.433333,301.52,4.0,2.0,6.0
4,sj,1990,26,1990-06-25,0.10245,0.146175,0.125571,0.123600,19.32,299.578571,...,17.484857,296.258571,296.089714,28.038095,28.234286,301.333333,301.66,5.0,4.0,2.0


In [13]:
# --- IQUITOS (IQ) ROLLING FEATURES ---

df_iq = df_clean[df_clean["city"] == "iq"].copy()

rolling_cols = [
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "reanalysis_min_air_temp_k",
    "station_min_temp_c",
]

# Rollings
for col in rolling_cols:
    df_iq[f"{col}_roll3"] = df_iq[col].rolling(window=3).mean()
    df_iq[f"{col}_roll5"] = df_iq[col].rolling(window=5).mean()

# Lag features
df_iq["cases_lag_1"] = df_iq["total_cases"].shift(1)
df_iq["cases_lag_2"] = df_iq["total_cases"].shift(2)
df_iq["cases_lag_3"] = df_iq["total_cases"].shift(3)

# NaN temizliği
df_iq = df_iq.dropna().reset_index(drop=True)

df_iq.head()


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_specific_humidity_g_per_kg_roll5,reanalysis_dew_point_temp_k_roll3,reanalysis_dew_point_temp_k_roll5,reanalysis_min_air_temp_k_roll3,reanalysis_min_air_temp_k_roll5,station_min_temp_c_roll3,station_min_temp_c_roll5,cases_lag_1,cases_lag_2,cases_lag_3
0,iq,2000,30,2000-07-29,0.328643,0.322129,0.254371,0.361043,62.76,296.432857,...,16.102000,294.125714,294.584000,290.900000,291.38,18.166667,19.20,0.0,0.0,0.0
1,iq,2000,31,2000-08-05,0.205529,0.190757,0.231671,0.255314,16.24,297.191429,...,15.456000,292.826667,293.892286,289.533333,290.46,16.933333,18.46,0.0,0.0,0.0
2,iq,2000,32,2000-08-12,0.312486,0.329986,0.380586,0.387271,89.37,297.320000,...,15.145714,293.168095,293.584857,290.700000,290.66,18.666667,18.28,0.0,0.0,0.0
3,iq,2000,33,2000-08-19,0.384133,0.392240,0.341780,0.382750,42.08,297.627143,...,14.814857,293.180000,293.258857,290.400000,290.26,19.133333,18.24,0.0,0.0,0.0
4,iq,2000,34,2000-08-26,0.408157,0.322157,0.406714,0.302714,49.22,298.238571,...,14.817429,293.572857,293.280286,291.333333,290.80,19.800000,19.10,0.0,0.0,0.0


In [14]:
import numpy as np

def add_seasonal_features(df_city):
    week = df_city["weekofyear"]
    df_city["week_sin"] = np.sin(2 * np.pi * week / 52)
    df_city["week_cos"] = np.cos(2 * np.pi * week / 52)
    return df_city

df_sj = add_seasonal_features(df_sj)
df_iq = add_seasonal_features(df_iq)

df_sj[["weekofyear", "week_sin", "week_cos"]].head()

# weekofyear = 1 ve 52 matematiksel olarak birbirine çok yakın ama model için “1 ve 52” gibi gözüküyor.
# Bunu düzeltmek için sin/cos ile dönüştürebiliriz.
# Hafta numarasının döngüsel yapısını yakalamak için weekofyear değişkenini sinüs ve kosinüs dönüşümleriyle encode ettik.


,weekofyear,week_sin,week_cos
0,22,4.647232e-01,-0.885456
1,23,3.546049e-01,-0.935016
2,24,2.393157e-01,-0.970942
3,25,1.205367e-01,-0.992709
4,26,-3.216245e-16,-1.000000


In [15]:
def add_temp_range(df_city):
    df_city["station_temp_range"] = df_city["station_max_temp_c"] - df_city["station_min_temp_c"]
    return df_city

df_sj = add_temp_range(df_sj)
df_iq = add_temp_range(df_iq)

df_sj[["station_min_temp_c", "station_max_temp_c", "station_temp_range"]].head()

# Sıcaklık aralığı (gün içi dalgalanma)
# Isı farkı → sivrisinek aktivitesi için önemli olabilir.


,station_min_temp_c,station_max_temp_c,station_temp_range
0,23.9,35.0,11.1
1,23.9,34.4,10.5
2,23.3,32.2,8.9
3,22.8,33.9,11.1
4,22.8,33.9,11.1


In [16]:
def add_temp_humidity_interaction(df_city):
    df_city["temp_humidity_interaction"] = (
        df_city["reanalysis_avg_temp_k"] * df_city["reanalysis_specific_humidity_g_per_kg"]
    )
    return df_city

df_sj = add_temp_humidity_interaction(df_sj)
df_iq = add_temp_humidity_interaction(df_iq)

df_sj[["reanalysis_avg_temp_k", 
       "reanalysis_specific_humidity_g_per_kg", 
       "temp_humidity_interaction"]].head()

# Nem tek başına, sıcaklık tek başına yetmeyebilir.
# Virüs için asıl önemli olan “sıcak ve nemli” ortam → bunu çarpım ile verebiliriz.

,reanalysis_avg_temp_k,reanalysis_specific_humidity_g_per_kg,temp_humidity_interaction
0,299.664286,17.210000,5157.222357
1,299.764286,17.212857,5159.799827
2,299.221429,17.234286,5156.867592
3,299.528571,17.977143,5384.667918
4,299.557143,17.790000,5329.121571


In [17]:
# Zaman temelli split:
# ✔ Veri kronolojik
# ✔ Test seti = SON 20%
# ✔ Train = önceki 80%
df_sj.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 40 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   city                                         932 non-null    object 
 1   year                                         932 non-null    int64  
 2   weekofyear                                   932 non-null    int64  
 3   week_start_date                              932 non-null    object 
 4   ndvi_ne                                      932 non-null    float64
 5   ndvi_nw                                      932 non-null    float64
 6   ndvi_se                                      932 non-null    float64
 7   ndvi_sw                                      932 non-null    float64
 8   precipitation_amt_mm                         932 non-null    float64
 9   reanalysis_air_temp_k                        932 non-null    float64
 10  re

In [18]:
# San Juan subset zaten hazır: df_sj

split_index = int(len(df_sj) * 0.8)
train_sj = df_sj.iloc[:split_index]
test_sj  = df_sj.iloc[split_index:]

# Hedef (label)
y_train_sj = train_sj["total_cases"]
y_test_sj  = test_sj["total_cases"]

# Özelliklerden city, week_start_date ve total_cases'i çıkar
drop_cols = ["total_cases", "city", "week_start_date"]

X_train_sj = train_sj.drop(columns=drop_cols)
X_test_sj  = test_sj.drop(columns=drop_cols)


In [19]:
# İquitos dataset
split_index = int(len(df_iq) * 0.8)
train_iq = df_iq.iloc[:split_index]
test_iq  = df_iq.iloc[split_index:]

y_train_iq = train_iq["total_cases"]
y_test_iq  = test_iq["total_cases"]

X_train_iq = train_iq.drop(columns=drop_cols)  # aynı drop_cols iş görür
X_test_iq  = test_iq.drop(columns=drop_cols)

y_test_iq = test_iq["total_cases"]


# split_index” mantığını çok basit anlatayım
# Diyelim ki 100 haftalık veri var
# %80 train → ilk 80 hafta
# %20 test → son 20 hafta
# O yüzden zaman temelli bölme yapıyoruz, rastgele değil
# iloc[:split_index] → dizinin başından split index’e kadar
# iloc[split_index:] → dizinin split index’ten sonuna kadar
# train_test_split() ZAMAN SERİSİ İÇİN KULLANILMAZ
# Karıştırır → leakage olur → model geleceği görür → yanlış sonuç
# ✔ YERİNE:
# Veri zaman sırasına göre olsun
# split_index = int(len(df) * 0.8)
# İlk %80 → TRAIN
# Son %20 → TEST
# Bu kadar.

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    random_state=42
)

model.fit(X_train_sj, y_train_sj)

pred_sj = model.predict(X_test_sj)
mae_sj = mean_absolute_error(y_test_sj, pred_sj)

mae_sj


6.826803305727302

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model_iq = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    random_state=42
)

model_iq.fit(X_train_iq, y_train_iq)

# 5) Tahmin & MAE
pred_iq = model_iq.predict(X_test_iq)
mae_iq = mean_absolute_error(y_test_iq, pred_iq)

mae_iq

4.566451821190541

In [ ]:
# Random Forest için basit hyperparameter tuning 
# manuel şekilde deneyerek en iyi parametreleri bulma 
# zaman serisi verisi için iyiymiş

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

def train_eval_rf(df_city, params, split_ratio=0.8):
    """
    df_city : tek şehir (sj veya iq)
    params  : {'n_estimators': .., 'max_depth': .., 'min_samples_leaf': ..}
    """

    # 1) SADECE SAYISAL FEATURE'LAR
    drop_cols = ["total_cases", "log_cases"]  # varsa log_cases'i de çıkar
    num_cols = df_city.select_dtypes(include=[np.number]).columns
    feature_cols = [c for c in num_cols if c not in drop_cols]

    # 2) Zaman sırasına göre train / test
    split_index = int(len(df_city) * split_ratio)
    train = df_city.iloc[:split_index]
    test  = df_city.iloc[split_index:]

    X_train = train[feature_cols]
    y_train = train["total_cases"]
    X_test  = test[feature_cols]
    y_test  = test["total_cases"]

    # 3) Model
    model = RandomForestRegressor(
        n_estimators=params["n_estimators"],
        max_depth=params["max_depth"],
        min_samples_leaf=params.get("min_samples_leaf", 1),
        random_state=42,
        n_jobs=-1,
    )

    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)

    return mae, model


In [23]:
# Hyperparameter kombinasyonları, Her parametre kombinasyonunu tek tek deniyor

param_grid = [
    {"n_estimators": 200, "max_depth": 10, "min_samples_leaf": 1},
    {"n_estimators": 300, "max_depth": 12, "min_samples_leaf": 1},
    {"n_estimators": 400, "max_depth": 14, "min_samples_leaf": 1},
    {"n_estimators": 400, "max_depth": 12, "min_samples_leaf": 2},
]

results_sj = []
results_iq = []

print("=== San Juan (sj) RF tuning ===")
for p in param_grid:
    mae, _ = train_eval_rf(df_sj, p)
    results_sj.append((p, mae))
    print(p, " -> MAE:", mae)

print("\n=== Iquitos (iq) RF tuning ===")
for p in param_grid:
    mae, _ = train_eval_rf(df_iq, p)
    results_iq.append((p, mae))
    print(p, " -> MAE:", mae)

# En iyi kombinasyonları bul
best_params_sj, best_mae_sj_rf = min(results_sj, key=lambda x: x[1])
best_params_iq, best_mae_iq_rf = min(results_iq, key=lambda x: x[1])

print("\nEn iyi SJ RF:", best_params_sj, "MAE:", best_mae_sj_rf)
print("En iyi IQ RF:", best_params_iq, "MAE:", best_mae_iq_rf)


=== San Juan (sj) RF tuning ===
Şehir: sj
{'n_estimators': 200, 'max_depth': 10, 'min_samples_leaf': 1}  -> MAE: 6.835476623543054
Şehir: sj
{'n_estimators': 300, 'max_depth': 12, 'min_samples_leaf': 1}  -> MAE: 6.8268033057273
Şehir: sj
{'n_estimators': 400, 'max_depth': 14, 'min_samples_leaf': 1}  -> MAE: 6.837949543654768
Şehir: sj
{'n_estimators': 400, 'max_depth': 12, 'min_samples_leaf': 2}  -> MAE: 6.925337925147608

=== Iquitos (iq) RF tuning ===
Şehir: iq
{'n_estimators': 200, 'max_depth': 10, 'min_samples_leaf': 1}  -> MAE: 4.619481368892068
Şehir: iq
{'n_estimators': 300, 'max_depth': 12, 'min_samples_leaf': 1}  -> MAE: 4.56645182119054
Şehir: iq
{'n_estimators': 400, 'max_depth': 14, 'min_samples_leaf': 1}  -> MAE: 4.532552832777934
Şehir: iq
{'n_estimators': 400, 'max_depth': 12, 'min_samples_leaf': 2}  -> MAE: 4.511001709404989

En iyi SJ RF: {'n_estimators': 300, 'max_depth': 12, 'min_samples_leaf': 1} MAE: 6.8268033057273
En iyi IQ RF: {'n_estimators': 400, 'max_depth': 

In [24]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error


In [25]:
def train_eval_poly(df_city, feature_cols, degree=2, alpha=1.0, split_ratio=0.8):
    """
    feature_cols : sadece seçtiğimiz en anlamlı kolonlar (korelasyondan gelenler + lag)
    degree      : polinom derecesi (k)
    alpha       : Ridge regularization güç parametresi
    """
    split_index = int(len(df_city) * split_ratio)
    train = df_city.iloc[:split_index]
    test  = df_city.iloc[split_index:]

    X_train = train[feature_cols]
    y_train = train["total_cases"]
    X_test  = test[feature_cols]
    y_test  = test["total_cases"]

    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("poly", PolynomialFeatures(degree=degree, include_bias=False)),
        ("model", Ridge(alpha=alpha))
    ])

    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    mae = mean_absolute_error(y_test, pred)

    return mae, pipe


In [26]:
poly_features_sj = [
    "weekofyear",
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "station_avg_temp_c",
    "reanalysis_max_air_temp_k",
    
]

mae_poly_sj, model_poly_sj = train_eval_poly(
    df_sj,
    feature_cols=poly_features_sj,
    degree=2,    # k=2: hem ana etkiler hem etkileşimler, çok patlamıyor
    alpha=1.0
)

print("San Juan Polynomial Regression MAE:", mae_poly_sj)


San Juan Polynomial Regression MAE: 24.384992309735114


In [27]:
poly_features_iq = [
    "weekofyear",
    "year",
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "reanalysis_min_air_temp_k",
    "station_min_temp_c",
]

mae_poly_iq, model_poly_iq = train_eval_poly(
    df_iq,
    feature_cols=poly_features_iq,
    degree=2,
    alpha=0.5
)

print("Iquitos Polynomial Regression MAE:", mae_poly_iq)


Iquitos Polynomial Regression MAE: 8.416651859754346


In [28]:
import pandas as pd

comparison = pd.DataFrame({
    "City": ["San Juan", "Iquitos", "San Juan", "Iquitos"],
    "Model": ["RandomForest", "RandomForest", "Polynomial(k=2)", "Polynomial(k=2)"],
    "MAE": [best_mae_sj_rf, best_mae_iq_rf, mae_poly_sj, mae_poly_iq]
})

comparison


,City,Model,MAE
0,San Juan,RandomForest,6.826803
1,Iquitos,RandomForest,4.511002
2,San Juan,Polynomial(k=2),24.384992
3,Iquitos,Polynomial(k=2),8.416652


In [29]:
# ---- 1) En önemli 5 feature ----
top5_sj = [
    "weekofyear",
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "station_avg_temp_c",
    "reanalysis_max_air_temp_k"
]

top5_iq = [
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "reanalysis_min_air_temp_k",
    "station_min_temp_c",
    "year"
]


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def rf_with_selected_features(df, selected_cols, split_index):
    X = df[selected_cols]
    y = df["total_cases"]

    X_train = X.iloc[:split_index]
    y_train = y.iloc[:split_index]

    X_test = X.iloc[split_index:]
    y_test = y.iloc[split_index:]

    model = RandomForestRegressor(
        n_estimators=300,
        max_depth=12,
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    mae = mean_absolute_error(y_test, preds)
    return mae


In [30]:
# Time-series split index (80% train, 20% test)
split_index_sj = int(len(df_sj) * 0.8)
split_index_iq = int(len(df_iq) * 0.8)

split_index_sj, split_index_iq


(745, 412)

In [31]:
mae_sj_5 = rf_with_selected_features(df_sj, top5_sj, split_index_sj)
print("San Juan → RF (5 feature) MAE:", mae_sj_5)


San Juan → RF (5 feature) MAE: 26.60577768094125


In [32]:
mae_iq_5 = rf_with_selected_features(df_iq, top5_iq, split_index_iq)
print("Iquitos → RF (5 feature) MAE:", mae_iq_5)


Iquitos → RF (5 feature) MAE: 10.08302772745902


In [33]:
# Zaman serisi (time-series) için klasik GridSearchCV veya RandomizedSearchCV kullanmak YANLIŞTIR. 
#Ama cv=5 (default) → yanlış
# ✔ cv=TimeSeriesSplit(...) → doğru GridSearchCV(model, params, cv=TimeSeriesSplit(...))

# RandomizedSearchCV de aynı şekilde kullanılabilir
# ✖ shuffle=True → yanlış
# ✔ cv=TimeSeriesSplit → doğru



In [34]:
# =======================
# XGBoost için feature listeleri (AYRI)
# =======================

drop_cols = ["total_cases", "city", "week_start_date"]

# SJ kendi kolonları
feature_cols_sj = [c for c in df_sj.columns if c not in drop_cols]

# IQ kendi kolonları
feature_cols_iq = [c for c in df_iq.columns if c not in drop_cols]

len(feature_cols_sj), len(feature_cols_iq)


# =======================
# SJ ve IQ için zaman serisi train/test bölümü (80% / 20%)
# =======================

split_index_sj = int(len(df_sj) * 0.8)
split_index_iq = int(len(df_iq) * 0.8)

split_index_sj, split_index_iq


# =======================
# XGBoost modeli + MAE hesaplayan fonksiyon
# =======================

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

def train_eval_xgb(df_city, feature_cols, split_index, params):
    """
    Tek şehir için:
    - Time-series split
    - XGBoost modeli kurulumu
    - MAE hesabı
    """
    train_city = df_city.iloc[:split_index].copy()
    test_city  = df_city.iloc[split_index:].copy()
    
    X_train = train_city[feature_cols]
    y_train = train_city["total_cases"]
    
    X_test  = test_city[feature_cols]
    y_test  = test_city["total_cases"]
    
    model = XGBRegressor(
        objective="reg:squarederror",
        random_state=42,
        n_estimators=params.get("n_estimators", 400),
        learning_rate=params.get("learning_rate", 0.05),
        max_depth=params.get("max_depth", 6),
        subsample=params.get("subsample", 0.9),
        colsample_bytree=params.get("colsample_bytree", 0.9),
        reg_lambda=params.get("reg_lambda", 1.0),
        reg_alpha=params.get("reg_alpha", 0.0),
        tree_method="hist",
        n_jobs=-1
    )
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    mae  = mean_absolute_error(y_test, pred)
    
    return mae, model


# =======================
# Parametre seti
# =======================

xgb_params = {
    "n_estimators": 500,
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "reg_lambda": 1.0,
    "reg_alpha": 0.0,
}


# =======================
# MODELLERİ ÇALIŞTIR
# =======================

mae_sj_xgb, model_sj_xgb = train_eval_xgb(df_sj, feature_cols_sj, split_index_sj, xgb_params)
mae_iq_xgb, model_iq_xgb = train_eval_xgb(df_iq, feature_cols_iq, split_index_iq, xgb_params)

mae_sj_xgb, mae_iq_xgb


(7.536930561065674, 5.173644065856934)

In [35]:
import random

# XGBoost için arama uzayı (makul ve küçük bir alan)
param_space = {
    "n_estimators":      [200, 400, 600],
    "max_depth":         [3, 4, 5, 6],
    "learning_rate":     [0.03, 0.05, 0.07, 0.10],
    "subsample":         [0.7, 0.9, 1.0],
    "colsample_bytree":  [0.7, 0.9, 1.0],
    "min_child_weight":  [1, 3, 5],
    "reg_lambda":        [0.0, 1.0, 5.0],
    "reg_alpha":         [0.0, 0.1, 0.5],
}


In [36]:
import pandas as pd

# Param uzayından rastgele N tane kombinasyon seçelim
def sample_param_list(space, n_samples=30, seed=42):
    random.seed(seed)
    keys = list(space.keys())
    combos = []
    for _ in range(n_samples):
        params = {k: random.choice(space[k]) for k in keys}
        combos.append(params)
    return combos

search_params = sample_param_list(param_space, n_samples=30)

results_sj = []
results_iq = []

for i, p in enumerate(search_params, start=1):
    print(f"\nDeneme {i} / {len(search_params)}  |  Parametreler: {p}")
    
    mae_sj, _ = train_eval_xgb(df_sj, feature_cols_sj, split_index_sj, p)
    mae_iq, _ = train_eval_xgb(df_iq, feature_cols_iq, split_index_iq, p)
    
    results_sj.append({"city": "sj", "mae": mae_sj, **p})
    results_iq.append({"city": "iq", "mae": mae_iq, **p})

print("\n=== En iyi 5 kombinasayon (San Juan) ===")
results_sj_df = pd.DataFrame(results_sj).sort_values("mae").head(5)
display(results_sj_df)

print("\n=== En iyi 5 kombinasayon (Iquitos) ===")
results_iq_df = pd.DataFrame(results_iq).sort_values("mae").head(5)
display(results_iq_df)




Deneme 1 / 30  |  Parametreler: {'n_estimators': 600, 'max_depth': 3, 'learning_rate': 0.03, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 1, 'reg_lambda': 0.0, 'reg_alpha': 0.0}

Deneme 2 / 30  |  Parametreler: {'n_estimators': 600, 'max_depth': 3, 'learning_rate': 0.03, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 1, 'reg_lambda': 0.0, 'reg_alpha': 0.0}

Deneme 3 / 30  |  Parametreler: {'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.03, 'subsample': 1.0, 'colsample_bytree': 0.7, 'min_child_weight': 5, 'reg_lambda': 5.0, 'reg_alpha': 0.5}

Deneme 4 / 30  |  Parametreler: {'n_estimators': 600, 'max_depth': 6, 'learning_rate': 0.05, 'subsample': 0.9, 'colsample_bytree': 1.0, 'min_child_weight': 3, 'reg_lambda': 0.0, 'reg_alpha': 0.0}

Deneme 5 / 30  |  Parametreler: {'n_estimators': 600, 'max_depth': 6, 'learning_rate': 0.07, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 1, 'reg_lambda': 1.0, 'reg_alpha': 0.0}

Deneme 6 / 30 

,city,mae,n_estimators,max_depth,learning_rate,subsample,colsample_bytree,min_child_weight,reg_lambda,reg_alpha
11,sj,6.635477,400,5,0.05,1.0,0.9,1,0.0,0.0
17,sj,6.688135,400,3,0.03,1.0,1.0,3,5.0,0.1
20,sj,6.731521,200,4,0.07,0.7,1.0,5,0.0,0.5
22,sj,6.783543,200,3,0.03,1.0,0.9,1,5.0,0.0
14,sj,6.823045,200,5,0.10,1.0,0.9,3,0.0,0.0



=== En iyi 5 kombinasayon (Iquitos) ===


,city,mae,n_estimators,max_depth,learning_rate,subsample,colsample_bytree,min_child_weight,reg_lambda,reg_alpha
3,iq,4.609147,600,6,0.05,0.9,1.0,3,0.0,0.0
25,iq,4.616080,600,6,0.03,0.7,0.7,1,1.0,0.0
21,iq,4.660561,400,6,0.03,0.7,0.9,3,0.0,0.0
24,iq,4.671416,600,4,0.07,0.9,1.0,5,1.0,0.1
8,iq,4.730450,400,3,0.05,0.7,0.9,3,1.0,0.5


In [37]:
param_names = list(param_space.keys())

best_params_sj = results_sj_df.iloc[0][param_names].to_dict()
best_params_iq = results_iq_df.iloc[0][param_names].to_dict()

print("San Juan için en iyi XGB param:", best_params_sj)
print("Iquitos için en iyi XGB param:", best_params_iq)

# 🔴 BURADA feature_cols DEĞİL, SJ ve IQ için AYRI LİSTELER KULLAN
best_mae_sj_xgb, best_model_sj_xgb = train_eval_xgb(
    df_sj, feature_cols_sj, split_index_sj, best_params_sj
)

best_mae_iq_xgb, best_model_iq_xgb = train_eval_xgb(
    df_iq, feature_cols_iq, split_index_iq, best_params_iq
)

print("\nFINAL XGBoost (SJ) MAE:", best_mae_sj_xgb)
print("FINAL XGBoost (IQ) MAE:", best_mae_iq_xgb)


San Juan için en iyi XGB param: {'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.05, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 1, 'reg_lambda': 0.0, 'reg_alpha': 0.0}
Iquitos için en iyi XGB param: {'n_estimators': 600, 'max_depth': 6, 'learning_rate': 0.05, 'subsample': 0.9, 'colsample_bytree': 1.0, 'min_child_weight': 3, 'reg_lambda': 0.0, 'reg_alpha': 0.0}

FINAL XGBoost (SJ) MAE: 6.635477066040039
FINAL XGBoost (IQ) MAE: 4.609147071838379


In [38]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import numpy as np

def cv_xgb_timeseries(df_city, feature_cols, params, n_splits=5):
    X = df_city[feature_cols].values
    y = df_city["total_cases"].values

    tscv = TimeSeriesSplit(n_splits=n_splits)

    maes = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X), 1):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = XGBRegressor(
            objective="reg:squarederror",
            random_state=42,
            **params
        )

        model.fit(X_train, y_train)
        pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, pred)
        maes.append(mae)

        print(f"Fold {fold} MAE: {mae:.4f}")

    print(f"\nOrtalama CV MAE: {np.mean(maes):.4f}")
    return np.mean(maes), maes


In [39]:
feature_cols = [c for c in df_sj.columns 
                if c not in ["total_cases", "city", "week_start_date"]]

best_params_sj = {
    "n_estimators": 400,
    "max_depth": 3,
    "learning_rate": 0.03,
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "min_child_weight": 3,
    "reg_lambda": 5.0,
    "reg_alpha": 0.1,
}

cv_mae_sj, fold_maes_sj = cv_xgb_timeseries(df_sj, feature_cols, best_params_sj, n_splits=5)


Fold 1 MAE: 28.3747
Fold 2 MAE: 13.8215
Fold 3 MAE: 7.6291
Fold 4 MAE: 4.5859
Fold 5 MAE: 7.3132

Ortalama CV MAE: 12.3449


In [40]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer, mean_absolute_error

X_sj = df_sj[feature_cols].values
y_sj = df_sj["total_cases"].values

tscv = TimeSeriesSplit(n_splits=5)

param_dist = {
    "n_estimators": [300, 400, 500, 600],
    "max_depth": [2, 3, 4],
    "learning_rate": [0.01, 0.03, 0.05],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "min_child_weight": [1, 3, 5],
    "reg_lambda": [0.0, 1.0, 5.0],
    "reg_alpha": [0.0, 0.1, 0.5],
}

xgb = XGBRegressor(objective="reg:squarederror", random_state=42)

search_sj = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=20,
    cv=tscv,                       # 🔴 burada TimeSeriesSplit kullanıyoruz
    scoring=make_scorer(mean_absolute_error, greater_is_better=False),
    n_jobs=-1,
    verbose=2,
    random_state=42,
)

search_sj.fit(X_sj, y_sj)

print("En iyi parametreler (SJ):", search_sj.best_params_)
print("En iyi CV MAE (SJ):", -search_sj.best_score_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
En iyi parametreler (SJ): {'subsample': 0.8, 'reg_lambda': 0.0, 'reg_alpha': 0.1, 'n_estimators': 300, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.03, 'colsample_bytree': 1.0}
En iyi CV MAE (SJ): 12.00220365524292


In [41]:
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, make_scorer
from xgboost import XGBRegressor

# Aynı feature columns listesi IQ için de geçerli
X_iq = df_iq[feature_cols_iq]
y_iq = df_iq["total_cases"]

# TimeSeriesSplit (5 fold)
tscv = TimeSeriesSplit(n_splits=5)

# Aynı parametre uzayı
param_dist = {
    "n_estimators": [200, 300, 400, 500, 600],
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.03, 0.05, 0.07],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.7, 0.8, 0.9, 1.0],
    "min_child_weight": [1, 2, 3, 5],
    "reg_lambda": [0.0, 0.5, 1.0],
    "reg_alpha": [0.0, 0.1, 0.5]
}

xgb = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    tree_method="hist",
    n_jobs=-1
)

search_iq = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=20,
    cv=tscv,
    scoring=make_scorer(mean_absolute_error, greater_is_better=False),
    n_jobs=-1,
    verbose=2,
    random_state=42
)

search_iq.fit(X_iq, y_iq)

print("=== IQ En iyi parametreler ===")
print(search_iq.best_params_)
print("=== IQ En iyi CV MAE ===")
print(-search_iq.best_score_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
=== IQ En iyi parametreler ===
{'subsample': 0.7, 'reg_lambda': 1.0, 'reg_alpha': 0.5, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.7}
=== IQ En iyi CV MAE ===
4.938499832153321


In [42]:
best_params_iq = search_iq.best_params_

# Train / Test split (80/20) — SJ ile aynı mantık
split_index_iq = int(len(df_iq) * 0.8)

train_iq = df_iq.iloc[:split_index_iq].copy()
test_iq  = df_iq.iloc[split_index_iq:].copy()

X_train_iq = train_iq[feature_cols_iq]
y_train_iq = train_iq["total_cases"]

X_test_iq = test_iq[feature_cols_iq]
y_test_iq = test_iq["total_cases"]


model_iq_final = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    tree_method="hist",
    n_jobs=-1,
    **best_params_iq
)

model_iq_final.fit(X_train_iq, y_train_iq)
pred_iq = model_iq_final.predict(X_test_iq)

final_mae_iq = mean_absolute_error(y_test_iq, pred_iq)

print("FINAL XGBoost (IQ) MAE:", final_mae_iq)


FINAL XGBoost (IQ) MAE: 4.638535022735596


In [43]:
import pandas as pd
import numpy as np
import xgboost as xgb

# =====================================
# 1) Veriyi oku
# =====================================
PATH = "Data/"  # klasör ismin farklıysa burayı değiştir

features_train = pd.read_csv(PATH + "dengue_features_train.csv")
labels_train   = pd.read_csv(PATH + "dengue_labels_train.csv")
features_test  = pd.read_csv(PATH + "dengue_features_test.csv")
submission     = pd.read_csv(PATH + "submission_format.csv")

# train = feature + label birleştir
train = features_train.merge(labels_train, on=["city", "year", "weekofyear"])
test  = features_test.copy()

# =====================================
# 2) Feature engineering (train + test)
# =====================================

rolling_cols = [
    "reanalysis_specific_humidity_g_per_kg",
    "reanalysis_dew_point_temp_k",
    "station_avg_temp_c",
    "reanalysis_max_air_temp_k",
]

def add_features(df):
    # Zaman sırasına göre sırala
    df = df.sort_values(["city", "year", "weekofyear"]).reset_index(drop=True)

    # Eksik değer doldurma
    df = df.fillna(method="ffill").fillna(method="bfill")

    # Şehir bazlı rolling + ek featurelar
    for city_code in ["sj", "iq"]:
        mask = df["city"] == city_code
        df_c = df.loc[mask].copy()

        # Rolling ortalamalar
        for col in rolling_cols:
            df_c[f"{col}_roll3"] = df_c[col].rolling(window=3, min_periods=1).mean()
            df_c[f"{col}_roll5"] = df_c[col].rolling(window=5, min_periods=1).mean()

        # Sıcaklık aralığı
        df_c["station_temp_range"] = (
            df_c["station_max_temp_c"] - df_c["station_min_temp_c"]
        )

        # Sıcaklık * nem etkileşimi
        df_c["temp_humidity_interaction"] = (
            df_c["reanalysis_avg_temp_k"]
            * df_c["reanalysis_specific_humidity_g_per_kg"]
        )

        # Şehir için geri yaz
        df.loc[mask, df_c.columns] = df_c

    # Rolling’den gelen ufak NaN’leri yine dolduralım
    df = df.fillna(method="ffill").fillna(method="bfill")
    return df

train_fe = add_features(train)
test_fe  = add_features(test)

# =====================================
# 3) XGBoost – en iyi parametreler
#    (RandomizedSearch + TimeSeriesSplit ile bulduklarımız)
# =====================================

best_params_sj = {
    "n_estimators": 400,
    "max_depth": 3,
    "learning_rate": 0.03,
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "min_child_weight": 3,
    "reg_lambda": 5.0,
    "reg_alpha": 0.1,
}

best_params_iq = {
    "n_estimators": 600,
    "max_depth": 3,
    "learning_rate": 0.03,
    "subsample": 1.0,
    "colsample_bytree": 0.9,
    "min_child_weight": 1,
    "reg_lambda": 0.0,
    "reg_alpha": 0.0,
}

def fit_predict_city(city_code, params):
    # Şehir filtrele + zaman sırasına göre sırala
    train_c = train_fe[train_fe["city"] == city_code].sort_values(
        ["year", "weekofyear"]
    )
    test_c = test_fe[test_fe["city"] == city_code].sort_values(
        ["year", "weekofyear"]
    )

    y = train_c["total_cases"]

    # Sadece sayısal kolonları al, target'ı çıkar
    num_cols = train_c.select_dtypes(include=[np.number]).columns
    feature_cols = [c for c in num_cols if c != "total_cases"]

    X_train = train_c[feature_cols]
    X_test  = test_c[feature_cols]

    model = xgb.XGBRegressor(
        objective="reg:squarederror",
        tree_method="hist",
        random_state=42,
        **params
    )

    model.fit(X_train, y)

    preds = model.predict(X_test)
    preds = np.maximum(preds, 0)          # negatif olmasın
    preds = np.round(preds).astype(int)   # vaka sayısı tam sayı
    return preds

# Şehir bazlı tahminler
sj_preds = fit_predict_city("sj", best_params_sj)
iq_preds = fit_predict_city("iq", best_params_iq)

# =====================================
# 4) Submission dosyasını oluştur
#    (Kaggle sırası: önce tüm SJ, sonra tüm IQ)
# =====================================

all_preds = np.concatenate([sj_preds, iq_preds])
submission["total_cases"] = all_preds

out_path = PATH + "xgb_submission_best.csv"
submission.to_csv(out_path, index=False)

print("Submission hazır:", out_path)
print("İlk 5 satır:")
print(submission.head())


C:\Users\MONSTER\AppData\Local\Temp\ipykernel_16172\691283926.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill").fillna(method="bfill")
C:\Users\MONSTER\AppData\Local\Temp\ipykernel_16172\691283926.py:62: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill").fillna(method="bfill")
C:\Users\MONSTER\AppData\Local\Temp\ipykernel_16172\691283926.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="ffill").fillna(method="bfill")
C:\Users\MONSTER\AppData\Local\Temp\ipykernel_16172\691283926.py:62: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fi

Submission hazır: Data/xgb_submission_best.csv
İlk 5 satır:
  city  year  weekofyear  total_cases
0   sj  2008          18            6
1   sj  2008          19            5
2   sj  2008          20            7
3   sj  2008          21           17
4   sj  2008          22            9


In [4]:
from src.data import load_raw_data, build_train_dataframe, split_and_clean_by_city

train_features, train_labels, test_features = load_raw_data()  # Data otomatik
df = build_train_dataframe(train_features, train_labels)
df_sj, df_iq = split_and_clean_by_city(df)

df_sj.head()


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,...,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0,4
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,...,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6,5
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,...,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4,4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,...,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0,3
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,...,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8,6


In [5]:
from src.data import load_raw_data, build_train_dataframe, split_and_clean_by_city
from src.features import build_features_city_concat, get_feature_cols

train_features, train_labels, test_features = load_raw_data()
df = build_train_dataframe(train_features, train_labels)

# şehir bazında train ayır
df_sj, df_iq = split_and_clean_by_city(df)

# test'i şehir bazında ayır
test_sj = test_features[test_features["city"]=="sj"].sort_values(["year","weekofyear"]).reset_index(drop=True)
test_iq = test_features[test_features["city"]=="iq"].sort_values(["year","weekofyear"]).reset_index(drop=True)

# feature üret
tr_sj, te_sj = build_features_city_concat(df_sj, test_sj, include_cases_lags=False)
feat_cols_sj = get_feature_cols(tr_sj)

print("SJ train shape:", tr_sj.shape)
print("SJ test  shape:", te_sj.shape)
print("Feature count:", len(feat_cols_sj))

# uyum kontrolü
missing_in_test = [c for c in feat_cols_sj if c not in te_sj.columns]
print("Missing in test:", missing_in_test[:10], "count:", len(missing_in_test))


SJ train shape: (936, 64)
SJ test  shape: (260, 64)
Feature count: 59
Missing in test: [] count: 0
